# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# Transform original dataset and save it as csv file
import pandas as pd

file ='winequality-red.csv'
ds= pd.read_csv(file)
ds["quality"]=ds["quality"].apply(lambda s: 1 if s >= 7 else 0)	

ds.to_csv("winequality-red-transformated2.csv", index=False)


In [2]:
from azureml.core import Workspace, Datastore, Dataset


# get existing workspace
workspace = Workspace.from_config()

In [3]:
#Get transformed data

from azureml.core import Workspace, Datastore, Dataset

datastore_name = 'workspaceblobstore'

# get existing workspace
workspace = Workspace.from_config()
    
# retrieve an existing datastore in the workspace by name
datastore = Datastore.get(workspace, datastore_name)

# create a TabularDataset from file paths in datastore
datastore_paths = [(datastore, 'UI/08-08-2021_082514_UTC/winequality-red-transformated2.csv')]

ds_t = Dataset.Tabular.from_delimited_files(path=datastore_paths)



In [4]:
ds_t.take(10).to_pandas_dataframe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,0
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,0
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,0


In [3]:
#Get existed experiment

experiment_name = 'red-wine-quality'

experiment=Experiment(workspace, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(
    compute_target = 'natcluster001',
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric='accuracy',
    training_data= ds_t,
    label_column_name= 'quality',
    n_cross_validations= 2)

In [7]:
# Submit  experiment
wine_automl_run = experiment.submit(config=automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
red-wine-quality,AutoML_16b2242a-77dc-407b-ae2e-d37d27d77d00,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(wine_automl_run).show()

wine_automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
red-wine-quality,AutoML_16b2242a-77dc-407b-ae2e-d37d27d77d00,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+==========================

{'runId': 'AutoML_16b2242a-77dc-407b-ae2e-d37d27d77d00',
 'target': 'natcluster001',
 'status': 'Completed',
 'startTimeUtc': '2021-08-08T08:28:01.294286Z',
 'endTimeUtc': '2021-08-08T09:03:01.211331Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'natcluster001',
  'AMLSettingsJsonString': '{"path":null,"name":"red-wine-quality","subscription_id":"3dafc970-6015-4fba-b00d-39dc6a1b7521","resource_group":"nk_capstoneproject","workspace_name":"nk_project3","region":"westeurope","compute_target":"natcluster001","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"accuracy","task_type":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run, fitted_model = wine_automl_run.get_output()
print(best_run)
print(fitted_model)
print(wine_automl_run.get_metrics())

Run(Experiment: red-wine-quality,
Id: AutoML_16b2242a-77dc-407b-ae2e-d37d27d77d00_39,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('kneighborsclassifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan', metric_params=None, n_jobs=1, n_neighbors=5, p=2, weights='distance'))], verbose=False))], flatten_transform=None, weights=[0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.3333333333333333, 0.16666666666666666]))],
         verbose=False)
{'experiment_status': ['DatasetEvaluation', 'FeaturesGene

In [ ]:
# Save the best model
model = wine_automl_run.register_model(model_name='wine-best-automl-model2')
print(model.name, model.id, model.version, sep='\t')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [8]:
# Deploy model
from azureml.train.automl.run import AutoMLRun
run_id= 'AutoML_16b2242a-77dc-407b-ae2e-d37d27d77d00_39' #best run
best_run=AutoMLRun(experiment, run_id)
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
red-wine-quality,AutoML_16b2242a-77dc-407b-ae2e-d37d27d77d00_39,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
# Retrive model by name
model = Model(workspace, 'wine-best-automl-model2')


In [13]:
#Download score script

script_file_name ='score2.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score2.py')

In [15]:
inference_config=InferenceConfig( entry_script='score2.py', environment=best_run.get_environment())
aciconfig= AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
aci_service_name='red-wine-quality'
print(aci_service_name)
aci_service=Model.deploy(workspace, aci_service_name, [model], inference_config, aciconfig)

aci_service.wait_for_deployment(True)
print(aci_service.state)

red-wine-quality
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-09 12:36:09+00:00 Creating Container Registry if not exists.
2021-08-09 12:36:10+00:00 Registering the environment.
2021-08-09 12:36:11+00:00 Use the existing image.
2021-08-09 12:36:11+00:00 Generating deployment configuration.
2021-08-09 12:36:12+00:00 Submitting deployment to compute.
2021-08-09 12:36:20+00:00 Checking the status of deployment red-wine-quality..
2021-08-09 12:39:38+00:00 Checking the status of inference endpoint red-wine-quality.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [25]:
# Enable application insights

aci_service.update(enable_app_insights=True)

In [17]:
# Test service (1)
import pandas as pd
test_df=pd.read_csv('test_wine_model.csv')
test_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.3,0.65,0.0,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10


In [18]:
import json
import requests

test_sample=json.dumps({'data': test_df.to_dict(orient='records')})
response=aci_service.run(input_data=test_sample)
response

'{"result": [1]}'

In [19]:
# Test service (2)
%run endpoint.py

{"result": [0, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [31]:
#Test service (3)
%run endpoint-copy.py

{"result": [1, 0, 0, 1]}


In [22]:
# Get logs of the web service
print(aci_service.get_logs())

2021-08-09T12:39:29,657749305+00:00 - iot-server/run 
2021-08-09T12:39:29,661759645+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-09T12:39:29,663071858+00:00 - nginx/run 
2021-08-09T12:39:29,662836556+00:00 - rsyslog/run 
rsyslogd: /azureml-envs/azureml_fc92eee9a5613508afa12283dd0b27d8/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-09T12:39:29,823707464+00:00 - iot-server/finish 1 0
2021-08-09T12:39:29,825327980+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 41
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-08-09 12:39:31,356 | root | INFO | Starting up app insights client
logging socket was 

In [12]:
workspace = Workspace.from_config()
service=Webservice(workspace, name='red-wine-quality')

In [10]:
model_name= best_run.properties['model_name']
model_name

'AutoML16b2242a739'